In [1]:
import os.path as osp
from time import time
from datetime import datetime, timedelta
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as T
from torchvision.datasets import MNIST

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

/home/chris/.local/lib/python3.9/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
class FFNN(nn.Module):
    def __init__(self, in_shape, out_shape, p_d1=0.5, p_d2=0.4):
        super().__init__()
        fc1 = nn.Linear(in_shape, 64)
        a1  = nn.ReLU()
        d1  = nn.Dropout(p=p_d1)
        fc2 = nn.Linear(64, 32)
        a2  = nn.ReLU()
        d2  = nn.Dropout(p=p_d2)
        fc3 = nn.Linear(32, out_shape)
        
        # not applying log_softmax here, as it is applied later in 
        # the torch CCE loss
        
        self.nn = nn.Sequential(fc1, a1, d1, fc2, a2, d2, fc3)

    def forward(self, x):
        x = self.nn(x)
        return x

In [3]:
def train_mnist(config, epochs, checkpoint_dir=None, data_dir=None):
    # create model
    model = FFNN(784, 10, p_d1=config['p_d1'], p_d2=config['p_d2'])
    
    # load data and make a validation split
    transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,),(0.5)), 
                            T.Lambda(lambda x: torch.flatten(x))])
    dataset_train = MNIST(root='/data/', transform=transforms, train=True)

    train_samples = int(len(dataset_train) * 0.8)
    train_subset, val_subset = random_split(dataset_train,
                                           [train_samples, 
                                            len(dataset_train) - train_samples])
    # create dataloaders
    train_args = {'dataset':train_subset, 
                  'batch_size':config['batch_size'], 
                  'shuffle':True, 
                  'num_workers':8, 
                  'pin_memory':True}
    dataloader_train = torch.utils.data.DataLoader(**train_args)
    val_args  = {'dataset':val_subset, 
                  'batch_size':len(val_subset), 
                  'shuffle':False, 
                  'num_workers':8}
    dataloader_val  = torch.utils.data.DataLoader(**val_args) 
    
    # choose computation host device
    device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device_name)
    model.to(device)
    
    
    optimiser = torch.optim.SGD(params=model.parameters(), lr=config['lr'], momentum=0.9)
    f_loss = nn.CrossEntropyLoss()
    
    # training loop
    for n in range(epochs):
        total_loss = 0.0
        # optimisation
        model.train()
        for idx, (X, y) in enumerate(dataloader_train):
            X, y = X.to(device), y.to(device)
            optimiser.zero_grad()
            y_pred = model(X)
            loss = f_loss(y_pred, y)
            loss.backward()
            total_loss += loss.detach().cpu().item() / len(y) # normalise for batch size
            optimiser.step()
            
        # validation set metrics
        predictions, targets, val_losses = [], [], []
        model.eval()
        # we are adding the metrics tensor for each batch to a list,
        # then concatenating at the end to make one tensor with all samples
        for idx, (X, y) in enumerate(dataloader_val):
            with torch.no_grad():
                y_pred = model(X)
                predictions.append(y_pred.detach())
                targets.append(y)
                val_losses.append(f_loss(y_pred, y).cpu().item())

        predictions = torch.cat(predictions, dim=0)
        targets = torch.cat(targets, dim=0)
        predictions = torch.argmax(F.log_softmax(predictions, dim=1),dim=1)
        corrects = (predictions == targets).sum().item()
        wrongs = len(targets) - corrects
        val_accuracy = corrects / len(targets)
        val_loss = sum(val_losses) / float(len(val_losses))
        
        # save checkpoint
        with tune.checkpoint_dir(n) as checkpoint_dir:
            path = osp.join(checkpoint_dir, 'checkpoint')
            torch.save((model.state_dict(), optimiser.state_dict()), path)
            
        # report metric values back to main scheduler
        tune.report(loss=val_loss, accuracy=val_accuracy)
        
def test_accuracy(model, device='cpu'):
    transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,),(0.5)), 
                            T.Lambda(lambda x: torch.flatten(x))])
    dataset_test  = MNIST(root='/data/', transform=transforms, train=False)
    test_args  = {'dataset':dataset_test, 
                  'batch_size':len(dataset_test), 
                  'shuffle':False, 
                  'num_workers':8}
    dataloader_test  = torch.utils.data.DataLoader(**test_args) 
    
    model.eval()
    predictions, targets = [], []
    with torch.no_grad():
        for (X, y) in dataloader_test:
            y_pred = model(X)
            predictions.append(y_pred.detach())
            targets.append(y)

        predictions = torch.cat(predictions, dim=0)
        targets = torch.cat(targets, dim=0)
        predictions = torch.argmax(F.log_softmax(predictions, dim=1),dim=1)
        corrects = (predictions == targets).sum().item()
        wrongs = len(targets) - corrects
        test_accuracy = corrects / len(targets)
        
    return  test_accuracy

In [4]:
max_epochs = 20
num_samples = 10

config = {'lr':tune.loguniform(1e-4, 1e-1), 
          'batch_size':tune.choice([32, 64, 256, 512]), 
          'p_d1':tune.uniform(0.1,0.9), 
          'p_d2':tune.uniform(0.1,0.9)}


scheduler = ASHAScheduler(metric='loss', 
                        mode='min',
                        max_t=20, 
                        grace_period=1, 
                        reduction_factor=2)

reporter  = CLIReporter(metric_columns=['loss', 'accuracy', 'training_iteration'])

result = tune.run(partial(train_mnist, epochs=max_epochs),
                  resources_per_trial={'cpu':2}, 
                  config=config, 
                  num_samples=num_samples, 
                  scheduler=scheduler, 
                  progress_reporter=reporter)

best_trial = result.get_best_trial('loss', 'min', 'last')

2021-07-04 20:27:44,548	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265
2021-07-04 20:27:46,425	WARNING experiment.py:293 -- No name detected on trainable. Using DEFAULT.
2021-07-04 20:27:46,426	INFO registry.py:64 -- Detected unknown callable for trainable. Converting to class.
2021-07-04 20:27:46,723	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


== Status ==
Memory usage on this node: 1.9/7.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.98 GiB objects
Result logdir: /home/chris/ray_results/DEFAULT_2021-07-04_20-27-46
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+--------------+-------------+----------+----------+
| Trial name          | status   | loc   |   batch_size |          lr |     p_d1 |     p_d2 |
|---------------------+----------+-------+--------------+-------------+----------+----------|
| DEFAULT_e953d_00000 | PENDING  |       |           64 | 0.000359741 | 0.776499 | 0.805166 |
| DEFAULT_e953d_00001 | PENDING  |       |          256 | 0.0163747   | 0.120924 | 0.76933  |
| DEFAULT_e953d_00002 | PENDING  |       |          512 | 0.0015177   | 0.832719 | 0.173309 |
| DEFAULT_e953d_00003 | PENDING  |       |          512 | 0.

(pid=120442) /home/chris/.local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:174.)
(pid=120442)   return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
(pid=120447) /home/chris/.local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable be

== Status ==
Memory usage on this node: 2.8/7.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.98 GiB objects
Result logdir: /home/chris/ray_results/DEFAULT_2021-07-04_20-27-46
Number of trials: 10/10 (6 PENDING, 4 RUNNING)
+---------------------+----------+-------+--------------+-------------+----------+----------+
| Trial name          | status   | loc   |   batch_size |          lr |     p_d1 |     p_d2 |
|---------------------+----------+-------+--------------+-------------+----------+----------|
| DEFAULT_e953d_00000 | RUNNING  |       |           64 | 0.000359741 | 0.776499 | 0.805166 |
| DEFAULT_e953d_00001 | RUNNING  |       |          256 | 0.0163747   | 0.120924 | 0.76933  |
| DEFAULT_e953d_00002 | RUNNING  |       |          512 | 0.0015177   | 0.832719 | 0.173309 |
| DEFAULT_e953d_00003 | RUNNING  |       |      

(pid=120446) /home/chris/.local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:174.)
(pid=120446)   return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


...

Result for DEFAULT_e953d_00009:
  accuracy: 0.91725
  date: 2021-07-04_20-31-00
  done: false
  experiment_id: 6a950c29a43b4cb196f58d92aa0d6cbe
  hostname: chris-server
  iterations_since_restore: 12
  loss: 0.3101970851421356
  node_ip: 192.168.1.58
  pid: 128385
  should_checkpoint: true
  time_since_restore: 85.55218887329102
  time_this_iter_s: 5.0559961795806885
  time_total_s: 85.55218887329102
  timestamp: 1625427060
  timesteps_since_restore: 0
  training_iteration: 12
  trial_id: e953d_00009
  
== Status ==
Memory usage on this node: 2.1/7.7 GiB
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 16.000: -0.454297699034214 | Iter 8.000: -0.3302193880081177 | Iter 4.000: -0.6645342707633972 | Iter 2.000: -1.5008934140205383 | Iter 1.000: -2.177282691001892
Resources requested: 4.0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.98 GiB objects (0.0/2.0 CPU_group_fbe0b91c8b2458be202dda7d5353dad3, 0.0/2.0 CPU_group_168530008168d15bc49a2e2d936da2f9, 0.0/2.0 CPU_group_0_fbe0b91c8b2458be202

Result for DEFAULT_e953d_00008:
  accuracy: 0.9174166666666667
  date: 2021-07-04_20-31-13
  done: false
  experiment_id: 83644ace4c2c4ae2a7b3ba039255a8f9
  hostname: chris-server
  iterations_since_restore: 12
  loss: 0.3001546859741211
  node_ip: 192.168.1.58
  pid: 126983
  should_checkpoint: true
  time_since_restore: 117.51355981826782
  time_this_iter_s: 7.33526086807251
  time_total_s: 117.51355981826782
  timestamp: 1625427073
  timesteps_since_restore: 0
  training_iteration: 12
  trial_id: e953d_00008
  
Result for DEFAULT_e953d_00009:
  accuracy: 0.9254166666666667
  date: 2021-07-04_20-31-17
  done: false
  experiment_id: 6a950c29a43b4cb196f58d92aa0d6cbe
  hostname: chris-server
  iterations_since_restore: 15
  loss: 0.2665010094642639
  node_ip: 192.168.1.58
  pid: 128385
  should_checkpoint: true
  time_since_restore: 101.89608073234558
  time_this_iter_s: 5.32626485824585
  time_total_s: 101.89608073234558
  timestamp: 1625427077
  timesteps_since_restore: 0
  training_i

Result for DEFAULT_e953d_00008:
  accuracy: 0.9220833333333334
  date: 2021-07-04_20-31-28
  done: false
  experiment_id: 83644ace4c2c4ae2a7b3ba039255a8f9
  hostname: chris-server
  iterations_since_restore: 14
  loss: 0.2851559519767761
  node_ip: 192.168.1.58
  pid: 126983
  should_checkpoint: true
  time_since_restore: 131.62208127975464
  time_this_iter_s: 7.133996486663818
  time_total_s: 131.62208127975464
  timestamp: 1625427088
  timesteps_since_restore: 0
  training_iteration: 14
  trial_id: e953d_00008
  
Result for DEFAULT_e953d_00009:
  accuracy: 0.9264166666666667
  date: 2021-07-04_20-31-33
  done: false
  experiment_id: 6a950c29a43b4cb196f58d92aa0d6cbe
  hostname: chris-server
  iterations_since_restore: 18
  loss: 0.27444714307785034
  node_ip: 192.168.1.58
  pid: 128385
  should_checkpoint: true
  time_since_restore: 117.99564337730408
  time_this_iter_s: 5.699957370758057
  time_total_s: 117.99564337730408
  timestamp: 1625427093
  timesteps_since_restore: 0
  trainin

2021-07-04 20:31:43,549	INFO tune.py:549 -- Total run time: 237.13 seconds (236.96 seconds for the tuning loop).


Result for DEFAULT_e953d_00009:
  accuracy: 0.91875
  date: 2021-07-04_20-31-43
  done: true
  experiment_id: 6a950c29a43b4cb196f58d92aa0d6cbe
  hostname: chris-server
  iterations_since_restore: 20
  loss: 0.29735755920410156
  node_ip: 192.168.1.58
  pid: 128385
  should_checkpoint: true
  time_since_restore: 128.07539629936218
  time_this_iter_s: 4.778563499450684
  time_total_s: 128.07539629936218
  timestamp: 1625427103
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: e953d_00009
  
== Status ==
Memory usage on this node: 1.7/7.7 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 16.000: -0.2786775827407837 | Iter 8.000: -0.3302193880081177 | Iter 4.000: -0.6645342707633972 | Iter 2.000: -1.5008934140205383 | Iter 1.000: -2.177282691001892
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/3.97 GiB heap, 0.0/1.98 GiB objects (0.0/2.0 CPU_group_0_fbe0b91c8b2458be202dda7d5353dad3, 0.0/2.0 CPU_group_168530008168d15bc49a2e2d936da2f9, 0.0/2.0 CPU_group_0_168530008168d15b

In [5]:
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

Best trial config: {'lr': 0.016374686378100446, 'batch_size': 256, 'p_d1': 0.12092449662468381, 'p_d2': 0.7693301753948671}
Best trial final validation loss: 0.19369226694107056
Best trial final validation accuracy: 0.9528333333333333


In [6]:
best_trained_model = FFNN(784, 10, 
                          p_d1=best_trial.config['p_d1'], 
                          p_d2=best_trial.config['p_d2'])

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimiser_state = torch.load(osp.join(best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)
test_acc = test_accuracy(best_trained_model)
print("Best trial test set accuracy: {}".format(test_acc))

/home/chris/.local/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:174.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Best trial test set accuracy: 0.9535
